### Exercise 04 : Enrichment and transformations

In [1]:
import pandas as pd
import numpy as np

* read the JSON file that you saved in `ex02`

  * one of the columns has the float type, so let us define the format of it in
  pandas using `pd.options.display.float_format`: floats should be displayed with
  two decimals
  * there are values missing from the Model, do not do anything with them

In [2]:
df = pd.read_json("../data/auto.json")
pd.options.display.float_format = "{:.2f}".format
df

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,3200.00,Ford,Focus
1,E432XX77RUS,1,6500.00,Toyota,Camry
2,7184TT36RUS,1,2100.00,Ford,Focus
3,X582HE161RUS,2,2000.00,Ford,Focus
4,92918M178RUS,1,5700.00,Ford,Focus
...,...,...,...,...,...
720,Y163O8161RUS,2,1600.00,Ford,Focus
721,M0309X197RUS,1,22300.00,Ford,Focus
722,O673E8197RUS,2,600.00,Ford,Focus
723,8610T8154RUS,1,2000.00,Ford,Focus


* enrich the dataframe using a sample from that dataframe
  * create a sample with 200 new observations with `random_state = 21` 
    * the sample should not have new combinations of the `car number`,
    `make` and `model`, so the whole dataset will be consistent in these terms
    * there are no restrictions on the `refund` and `fines`, you can take
    any value
    from these columns at random and use it towards any car number
  * concatenate the sample with the initial dataframe to a new dataframe `concat_rows`

In [3]:
unic_cars = df[["CarNumber", "Make", "Model"]].drop_duplicates()
cars_samples = unic_cars.sample(n=200, random_state=21).reset_index(drop=True)
refund_samples = df["Refund"].sample(n=200, random_state=21).reset_index(drop=True)
fines_samples = df["Fines"].sample(n=200, random_state=21).reset_index(drop=True)
concat_rows = cars_samples
concat_rows["Refund"] = refund_samples
concat_rows["Fines"] = fines_samples
concat_rows = pd.concat([df, concat_rows], ignore_index=True)
concat_rows.count()

CarNumber    925
Refund       925
Fines        925
Make         912
Model        912
dtype: int64

* enrich the dataframe `concat_rows` by a new column with the data generated 
  * create a series with the name `Year` using random integers from `1980` to `2019`
  * use `np.random.seed(21)` before generating the years
  * concatenate the series with the dataframe and name it `fines`

In [4]:
np.random.seed(21)
year = pd.Series(np.random.randint(1980, 2020, size=len(concat_rows)))
year

0      1989
1      1995
2      1984
3      2015
4      2014
       ... 
920    1981
921    1992
922    2007
923    2005
924    1997
Length: 925, dtype: int64

In [5]:
fines = pd.concat([concat_rows, year.rename("Year")], axis=1)
fines

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995
2,7184TT36RUS,1,2100.00,Ford,Focus,1984
3,X582HE161RUS,2,2000.00,Ford,Focus,2015
4,92918M178RUS,1,5700.00,Ford,Focus,2014
...,...,...,...,...,...,...
920,8182XX154RUS,1,200.00,Ford,Focus,1981
921,T6418M116RUS,1,500.00,Ford,Focus,1992
922,E42377152RUS,2,4000.00,Ford,Focus,2007
923,C514X938RUS,2,1000.00,Ford,Focus,2005


* enrich the dataframe with the data from another dataframe
  * create a new dataframe with the car numbers and their owners
      * get the most popular surnames (**you can find the file [surname.json](datasets/surname.json) in the attachments**) in the US

In [6]:
df_surname = pd.read_json("../../datasets/surname.json")
df_surname.columns = df_surname.iloc[0]
df_surname.drop(0, inplace=True)
df_surname.reset_index(drop=True, inplace=True)
df_surname

,NAME,COUNT,RANK
0,ADAMS,427865,42
1,ALLEN,482607,33
2,ALVAREZ,233983,92
3,ANDERSON,784404,15
4,BAILEY,277845,72
...,...,...,...
95,WILLIAMS,1625252,3
96,WILSON,801882,14
97,WOOD,250715,84
98,WRIGHT,458980,35


* create a new series with the surnames (they should not have
      special characters like commas, brackets, etc.) from the data you gathered, the count
      should be equal to the number of unique car numbers using the sample
      (use `random_state = 21`)

In [7]:
uniq_carNumber = concat_rows["CarNumber"].unique()
surnames_samples = df_surname.sample(n=len(uniq_carNumber), replace=True, random_state=21)
surnames_samples

,NAME,COUNT,RANK
73,RICHARDSON,259798,80
79,ROSS,229368,98
56,MORGAN,286280,69
4,BAILEY,277845,72
48,LOPEZ,874523,12
...,...,...,...
9,CAMPBELL,386157,47
31,HALL,407076,45
5,BAKER,419586,44
20,DIAZ,347636,55


* create the dataframe owners with 2 columns: `CarNumber` and `SURNAME` 

In [8]:
car_series = pd.Series(uniq_carNumber).reset_index(drop=True)
name_series = surnames_samples["NAME"].reset_index(drop=True)
owners = pd.concat([car_series, name_series], axis=1)
owners.columns = ["CarNumber", "SURNAME"]
owners

,CarNumber,SURNAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
526,O136HO197RUS,CAMPBELL
527,O22097197RUS,HALL
528,M0309X197RUS,BAKER
529,O673E8197RUS,DIAZ


* append 5 more observations to the fines dataframe (come up with your own ideas of CarNumber, etc.)

In [18]:
my_observations = [["Q123W456T789RUS", 1, 1234, "Ford", "Focus", 1981], 
                   ["Y643H664G453RUS", 2, 5678, "Ford", "Focus", 1981], 
                   ["L993O993L993RUS", 1, 9930, "Ford", "Focus", 1981],
                   ["A111A111A111RUS", 2, 4321, "Ford", "Focus", 1981], 
                   ["B222B222B222RUS", 2, 5934, "Ford", "Focus", 1981]]
my_observations_df = pd.DataFrame(my_observations, columns=fines.columns)
fines = pd.concat([my_observations_df, fines], ignore_index=True)
fines.count()

CarNumber    935
Refund       935
Fines        935
Make         922
Model        922
Year         935
dtype: int64

* delete the dataframe last `20` observations from the owners and add 3 new
observations (they are not the same as those you add to the `fines` dataframe)

In [10]:
owners = owners.iloc[:-20]
owners

,CarNumber,SURNAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
506,T914CT197RUS,HERNANDEZ
507,E41977152RUS,BAKER
508,9464EX178RUS,MARTIN
509,O50197197RUS,WRIGHT


In [11]:
my_mew_owners = [["S777I777A777RUS", "ANNE"], ["S21C21H21O21L21RUS", "SBER"], ["LA123LA123LA123RUS", "LALALEND"]]
my_mew_owners_df = pd.DataFrame(my_mew_owners, columns=owners.columns)
owners = pd.concat([my_mew_owners_df, owners], ignore_index=True)
owners

,CarNumber,SURNAME
0,S777I777A777RUS,ANNE
1,S21C21H21O21L21RUS,SBER
2,LA123LA123LA123RUS,LALALEND
3,Y163O8161RUS,RICHARDSON
4,E432XX77RUS,ROSS
...,...,...
509,T914CT197RUS,HERNANDEZ
510,E41977152RUS,BAKER
511,9464EX178RUS,MARTIN
512,O50197197RUS,WRIGHT


* join both dataframes:
    * the new dataframe should have **only** the car numbers that exist in **both** dataframes 

In [12]:
join_df = fines.merge(owners, on="CarNumber", how="inner")
join_df

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,RICHARDSON
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROSS
2,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORGAN
3,X582HE161RUS,2,2000.00,Ford,Focus,2015,BAILEY
4,92918M178RUS,1,5700.00,Ford,Focus,2014,LOPEZ
...,...,...,...,...,...,...,...
897,8182XX154RUS,1,200.00,Ford,Focus,1981,SMITH
898,T6418M116RUS,1,500.00,Ford,Focus,1992,RUIZ
899,E42377152RUS,2,4000.00,Ford,Focus,2007,CARTER
900,C514X938RUS,2,1000.00,Ford,Focus,2005,RAMIREZ


* the new dataframe should have **all** the car numbers that exist in **both** dataframes 

In [13]:
join_df = fines.merge(owners, on="CarNumber", how="outer")
join_df

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,704687163RUS,2.00,1400.00,Ford,Focus,2004.00,ADAMS
1,704687163RUS,2.00,7000.00,Ford,Focus,2005.00,ADAMS
2,704787163RUS,2.00,2800.00,Ford,Focus,1992.00,MORGAN
3,704787163RUS,2.00,19200.00,Ford,Focus,1985.00,MORGAN
4,704987163RUS,2.00,8594.59,Ford,Focus,1985.00,MITCHELL
...,...,...,...,...,...,...,...
928,Y969O8197RUS,2.00,7800.00,Ford,Focus,1992.00,LOPEZ
929,Y973O8197RUS,2.00,8594.59,Ford,Focus,2005.00,YOUNG
930,Y973O8197RUS,1.00,34800.00,Ford,Focus,2003.00,YOUNG
931,Y973O8197RUS,1.00,69600.00,Ford,Focus,2017.00,YOUNG


* the new dataframe should have only the car numbers from the `fines` dataframe 

In [14]:
join_df = fines.merge(owners, on="CarNumber", how="left")
join_df

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Q123W456T789RUS,1,1234.00,Ford,Focus,1981,NaN
1,Y643H664G453RUS,2,5678.00,Ford,Focus,1981,NaN
2,L993O993L993RUS,1,9930.00,Ford,Focus,1981,NaN
3,A111A111A111RUS,2,4321.00,Ford,Focus,1981,NaN
4,B222B222B222RUS,2,5934.00,Ford,Focus,1981,NaN
...,...,...,...,...,...,...,...
925,8182XX154RUS,1,200.00,Ford,Focus,1981,SMITH
926,T6418M116RUS,1,500.00,Ford,Focus,1992,RUIZ
927,E42377152RUS,2,4000.00,Ford,Focus,2007,CARTER
928,C514X938RUS,2,1000.00,Ford,Focus,2005,RAMIREZ


* the new dataframe should have only the car numbers from the `owners` dataframe

In [15]:
join_df = fines.merge(owners, on="CarNumber", how="right")
join_df

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,S777I777A777RUS,NaN,NaN,NaN,NaN,NaN,ANNE
1,S21C21H21O21L21RUS,NaN,NaN,NaN,NaN,NaN,SBER
2,LA123LA123LA123RUS,NaN,NaN,NaN,NaN,NaN,LALALEND
3,Y163O8161RUS,2.00,3200.00,Ford,Focus,1989.00,RICHARDSON
4,Y163O8161RUS,2.00,1600.00,Ford,Focus,1980.00,RICHARDSON
...,...,...,...,...,...,...,...
900,E41977152RUS,2.00,2400.00,Ford,Focus,1989.00,BAKER
901,E41977152RUS,1.00,2200.00,Ford,Focus,2015.00,BAKER
902,9464EX178RUS,2.00,2100.00,Ford,Focus,1988.00,MARTIN
903,O50197197RUS,2.00,7800.00,Ford,Focus,1992.00,WRIGHT


* create a pivot table from the `fines` dataframe, but with all the years 

In [16]:
pivot = fines.pivot_table(columns="Year", index=["Make", "Model"], values="Fines", aggfunc="sum")
pivot 

Year                   1980      1981      1982     1983     1984      1985  \
Make       Model                                                              
Ford       Focus   49294.59 427486.17 166283.76 64300.00 95889.17 162683.76   
           Mondeo       NaN       NaN       NaN      NaN      NaN       NaN   
Skoda      Octavia 47000.00   8594.59   6900.00 12094.59  1100.00  10294.59   
Toyota     Camry   27600.00   8594.59       NaN  7200.00      NaN       NaN   
           Corolla      NaN       NaN   2000.00      NaN      NaN       NaN   
Volkswagen Golf    31700.00       NaN       NaN  8594.59   300.00  24000.00   
           Jetta        NaN       NaN       NaN      NaN      NaN       NaN   
           Passat       NaN   1600.00       NaN  3200.00 10000.00   5000.00   
           Touareg      NaN       NaN       NaN      NaN      NaN   5800.00   

Year                   1986      1987      1988      1989  ...      2010  \
Make       Model                                           ...             
Ford       Focus   93589.17 159900.00 108394.59 171700.00  ... 125489.17   
           Mondeo       NaN       NaN       NaN   8600.00  ...       NaN   
Skoda      Octavia   600.00   5200.00   8594.59  91400.00  ...  11694.59   
Toyota     Camry        NaN       NaN       NaN  22400.00  ...       NaN   
           Corolla  3000.00   8000.00       NaN   4000.00  ...  24000.00   
Volkswagen Golf         NaN   9300.00       NaN   5800.00  ...   8594.59   
           Jetta        NaN       NaN       NaN       NaN  ...       NaN   
           Passat  15000.00  12300.00       NaN   8594.59  ...   2800.00   
           Touareg      NaN       NaN       NaN       NaN  ...   6300.00   

Year                    2011      2012      2013      2014      2015     2016  \
Make       Model                                                                
Ford       Focus   103478.35 130400.00 157794.59 115083.76 195800.00 80089.17   
           Mondeo        NaN  34400.00       NaN   8594.59       NaN 46200.00   
Skoda      Octavia    500.00    500.00  12594.59    300.00  51694.59 19200.00   
Toyota     Camry         NaN   8594.59       NaN       NaN       NaN      NaN   
           Corolla   8594.59       NaN    300.00       NaN       NaN      NaN   
Volkswagen Golf       300.00       NaN       NaN       NaN   2300.00      NaN   
           Jetta         NaN       NaN   2100.00       NaN       NaN      NaN   
           Passat        NaN       NaN       NaN       NaN   8100.00  2100.00   
           Touareg       NaN       NaN       NaN   1300.00    500.00      NaN   

Year                    2017      2018     2019  
Make       Model                                 
Ford       Focus   271594.59 272789.17 76589.17  
           Mondeo        NaN       NaN      NaN  
Skoda      Octavia       NaN 156200.00  9500.00  
Toyota     Camry         NaN  15300.00 18100.00  
           Corolla   9600.00       NaN      NaN  
Volkswagen Golf          NaN       NaN      NaN  
           Jetta         NaN       NaN      NaN  
           Passat        NaN   8594.59  2300.00  
           Touareg       NaN       NaN      NaN  

[9 rows x 40 columns]

* save both the `fines` and `owners` dataframes to CSV files without an index

In [17]:
fines.to_csv("../data/fines.csv", index=False)
owners.to_csv("../data/owners.csv", index=False)